## Improve graph encoding for single/multiallele
### Ablation sequence masking feature

In [ ]:
!pip install openpyxl

In [ ]:
# Load in data 
import pandas as pd

# Load the Excel file
data = pd.read_excel('immuno_data_multi_allele.xlsx')

In [ ]:
import graphein
import random
import gc
import os
import torch
import networkx as nx
from graphein.ml.conversion import GraphFormatConvertor

In [ ]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.visualisation import plotly_protein_structure_graph
from graphein.protein.edges.distance import add_hydrogen_bond_interactions, add_peptide_bonds
from graphein.protein.graphs import construct_graph
from graphein.protein.features.nodes.amino_acid import amino_acid_one_hot
from graphein.protein.features.nodes.amino_acid import hydrogen_bond_acceptor
from graphein.protein.features.nodes.amino_acid import hydrogen_bond_donor
from graphein.protein.edges.intramolecular import hydrogen_bond, hydrophobic, peptide_bonds, van_der_waals
from graphein.utils.utils import annotate_edge_metadata

#from graphein.ml import ProteinGraphListDataset, GraphFormatConvertor
from graphein.ml.conversion import GraphFormatConvertor
from graphein.protein.edges.distance import add_aromatic_interactions, add_cation_pi_interactions, add_hydrophobic_interactions, add_ionic_interactions
from graphein.protein.edges.atomic import add_atomic_edges, add_bond_order
from graphein.protein.features.nodes.dssp import asa
from graphein.protein.features.nodes.amino_acid import expasy_protein_scale
from graphein.protein.features.nodes.geometry import add_sidechain_vector
from graphein.protein.subgraphs import extract_subgraph_by_sequence_position
import graphein.protein as gp
from graphein.protein.graphs import read_pdb_to_dataframe

In [ ]:
# Generate different edge constructions
new_edge_funcs = {"edge_construction_functions": [add_peptide_bonds, add_hydrogen_bond_interactions,
                                                  add_hydrophobic_interactions,add_ionic_interactions]
                  ,"node_metadata_functions": [amino_acid_one_hot,hydrogen_bond_acceptor,hydrogen_bond_donor]
                  #,"edge_metadata_functions" : [hydrogen_bond]
                  ,"granularity": "CA"
                  ,"exclude_waters": False}

# new_edge_funcs = {"edge_construction_functions": [add_peptide_bonds,add_hydrophobic_interactions]
#                   ,"node_metadata_functions": [amino_acid_one_hot,hydrogen_bond_acceptor,hydrogen_bond_donor]
#                   ,"granularity": "CA"
#                   ,"exclude_waters": False}

config = ProteinGraphConfig(**new_edge_funcs)

config.dict()

In [ ]:
# Load Alphafold PDB files
directory_path = "/home/ddz5/immunoGAT/MultiAllele/alpha_structure"

# Get all files in the directory
files = os.listdir(directory_path)
print(len([x for x in files if 'rank_1' in x]))
files = [x for x in files if 'rank_1' in x]
files = [x for x in files if '_Immuno' in x]

files = files
print(len(files))

In [ ]:
# Graphein does not appear to encode the peptide sequence uniquely, redefine an encoding sequence
enc_dict = {'GLY': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'SER': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'HIS': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'MET': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ARG': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'TYR': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 'PHE': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'THR': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 'VAL': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'PRO': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'GLU': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ILE': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ALA': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ASP': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'GLN': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'TRP': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'LYS': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'LEU': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ASN': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'CYS': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'MASK': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
def mask_sequence(seq_one_hot, percentage=10):
    total_len = len(seq_one_hot)
    mask_len = int(total_len * percentage / 100)

    mask_indices = random.sample(range(total_len), mask_len)

    masked_seq = seq_one_hot.clone()  # Create a copy of the original sequence
    for idx in mask_indices:
        masked_seq[idx] = torch.tensor(enc_dict['MASK'])

    return masked_seq

In [ ]:
data_dir = '/home/ddz5/immunoGAT/MultiAllele/alpha_structure/'

convertor = GraphFormatConvertor(src_format = 'nx', dst_format = 'pyg')
SEQUENCE_POSITIONS = range(1, 180)

pygs = []
g2s = []
peptide_order_list = []

indices = list(range(6333, len(files)))

for i, x in zip(indices, files[6333:]):
    file = data_dir + x
    data_label = x[:-4]
    
    g = construct_graph(config = config, path = file)

    g = gp.extract_subgraph_from_chains(g, ["A","B"])
    ga = gp.extract_subgraph_from_chains(g, ["A"])
    gb = gp.extract_subgraph_from_chains(g, ["B"])

    g2 = extract_subgraph_by_sequence_position(g, SEQUENCE_POSITIONS)
    #g2a = extract_subgraph_by_sequence_position(ga, SEQUENCE_POSITIONS)
    g2a = ga
    #g2b = extract_subgraph_by_sequence_position(gb, SEQUENCE_POSITIONS)
    g2b = gb

    g_pyg = convertor(g2)

    #READ IN PDBS FOR TROUBLE SHOOTING
    tdf = read_pdb_to_dataframe(file)

    tdfa = tdf[tdf['chain_id'] == 'A']
    tdfa = tdfa.drop_duplicates('residue_number')
    tdfa = tdfa[:179]
    sequence_a = tdfa['residue_name'].tolist()

    tdfb = tdf[tdf['chain_id'] == 'B']
    tdfb = tdfb.drop_duplicates('residue_number')
    sequence_b = tdfb['residue_name'].tolist()

    #add node features to each graph here
    n_hdonors = torch.tensor([d['hbond_donors'] for n, d in g2.nodes(data=True)])
    n_hacceptors = torch.tensor([d['hbond_acceptors'] for n, d in g2.nodes(data=True)])
    aa_one_hot = torch.tensor([d['amino_acid_one_hot'] for n, d in g2.nodes(data=True)])
    gphein_seq_a = torch.tensor([d['amino_acid_one_hot'] for n, d in g2a.nodes(data=True)])
    gphein_seq_b = torch.tensor([d['amino_acid_one_hot'] for n, d in g2b.nodes(data=True)])

    #amino acid one-hot encoding
    aa_one_hot_a = torch.tensor([enc_dict[x] for x in sequence_a])
    aa_one_hot_b = torch.tensor([enc_dict[x] for x in sequence_b])
    # print("my dict enc")
    # print('sequence a')
    # print(sequence_a[:9])
    # print(aa_one_hot_a[:9])
    # print(aa_one_hot_a.shape)
    # print('sequence b')
    # print(sequence_b)
    # print(aa_one_hot_b[:9])
    # print(aa_one_hot_b.shape)

    # print("graphein enc")
    # print('sequence a')
    # print(sequence_a[:9])
    # print(gphein_seq_a[:9])
    # print(gphein_seq_a.shape)
    # print('sequence b')
    # print(sequence_b)
    # print(gphein_seq_b[:9])
    # print(gphein_seq_a.shape)
    # print('done with test')

    # Apply masking with a X% chance of masking each amino acid (applied only to peptide, increase percentage argument (eg, to 50) to mask %50 of the peptide)
    masked_aa_one_hot_b = mask_sequence(aa_one_hot_b, percentage=0)
    aa_one_hot = torch.cat([aa_one_hot_a, masked_aa_one_hot_b], dim=0)

    #just some sanity check printing
    # print(aa_one_hot[:20])
    # print(aa_one_hot[-10:])
    # print(tdfa['residue_name'].tolist())
    # print(tdfb['residue_name'].tolist())
    # peptide_name = tdfb['residue_name'].tolist()
    # peptide_order_list.append(peptide_name)
    # print(len(tdfa['residue_name'].tolist()))
    # print(tdf)
    
    g2s.append(g2)

    # Use the masked amino acid one-hot encoding as node features, along with number of H-donors/acceptors
    node_feats = torch.cat([aa_one_hot, n_hdonors, n_hacceptors], dim =1)
    g_pyg.x = node_feats

    pygs.append(g_pyg)

    # Save the PyTorch graph to a file if desired

    torch.save(g_pyg, f'/home/ddz5/immunoGAT/MultiAllele/PyGs/graph_{i+1}' + data_label + '.pt')

    print('done creating graph for {}'.format(x))    
    
    # Delete temporary variables to free memory
    del x, g, g2, g_pyg
    gc.collect()  # Explicitly call garbage collecto